In [ ]:
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import tqdm

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# As per https://towardsdatascience.com/alexnet-8b05c5eb88d4
batch_size = 128
epochs = 90

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

alexnet = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=False)

# Changed device acquisition to use CUDA if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Move your model to the CUDA device
alexnet.to(device)

# Standard PyTorch training loop adapted for CUDA
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(), lr=0.001, momentum=0.9)


progress = tqdm.tqdm(range(epochs))
for epoch in progress:
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = alexnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
    train_loss = running_loss / len(trainloader.dataset)
    progress.set_description(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}")

print('Finished Training')



Files already downloaded and verified
Files already downloaded and verified


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Epoch 90/90, Train Loss: 0.0427: 100%|██████████| 90/90 [1:15:09<00:00, 50.10s/it]


Finished Training
Accuracy of Alexnet on the 10000 test images: 80 %


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Accuracy of the Pretrained Alexnet on the 10000 test images: 14 %


In [ ]:
checkpoint_path = './alexnet_checkpoint.pth'

checkpoint = {
    'epoch': epoch + 1,
    'model_state_dict': alexnet.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss,
}

torch.save(checkpoint, checkpoint_path)

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = alexnet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of Alexnet on the 10000 test images: {100 * correct // total} %')

Accuracy of Alexnet on the 10000 test images: 79 %


Pretrained Alexnet

In [ ]:
pt_alexnet = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
pt_alexnet.classifier.append(nn.Linear(in_features=1000, out_features=len(classes), bias=True))
pt_alexnet.to(device)
pt_alexnet.eval()

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = pt_alexnet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the Pretrained Alexnet on the 10000 test images: {100 * correct // total} %')


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


Accuracy of the Pretrained Alexnet on the 10000 test images: 8 %
